In [247]:
from huggingface_hub import notebook_login
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
#from google.colab import drive
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from thefuzz import fuzz
from thefuzz import process
import dedupe
import pandas_dedupe

In [271]:
excel_file_path = '/Users/anthonyhakim/DSI/good-food-purchasing/CGFP_bulk.xlsx'
directory_file_path = '/Users/anthonyhakim/DSI/good-food-purchasing/CGFP_bulk_holout_set.csv'
complete_food_data = pd.read_excel(excel_file_path)

In [272]:
complete_food_data, holdout_set = train_test_split(complete_food_data, test_size=0.2)

In [273]:
holdout_set.to_csv(directory_file_path, index=False)

In [308]:
complete_food_data.to_csv('/Users/anthonyhakim/DSI/good-food-purchasing/CGFP_bulk_training_set.csv')

In [274]:
btype_data = complete_food_data[['Product Type', 'Food Product Group', 'Product Name']]
btype_data.rename(columns={'Product Type': 'text', 'Food Product Group': 'food_category', 'Product Name': 'labels'}, inplace=True)
btype_data['labels'] = btype_data['labels'].str.lower().str.split(',').str[0].str.strip()
btype_data.dropna(inplace=True)
btype_data.reset_index(drop=True, inplace=True)
btype_data['labels'] = btype_data['labels'].str.replace('prequalified: ', '')

/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_28059/4145565903.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btype_data.rename(columns={'Product Type': 'text', 'Food Product Group': 'food_category', 'Product Name': 'labels'}, inplace=True)
/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_28059/4145565903.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btype_data['labels'] = btype_data['labels'].str.lower().str.split(',').str[0].str.strip()
/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_28059/4145565903.py:4: SettingWit

In [275]:
df = btype_data[btype_data['food_category'] == 'Produce'][['labels']]
df['value_counts_column'] = df['labels'].map(df['labels'].value_counts())
df = df.sort_values('value_counts_column', ascending=False).reset_index(drop=True)

In [276]:
food_category = btype_data['food_category'].unique()
food_category

array(['Condiments & Snacks', 'Produce', 'Milk & Dairy',
       'Bread, Grains & Legumes', 'Meat', 'Beverages', 'Meals', 'Seafood',
       'Non-Food'], dtype=object)

In [277]:
btype_data.labels.unique().__len__()

760

In [278]:
def group_labels(dataframe: pd.DataFrame, category: str, category_label: str, labels_column = 'labels') -> dict:
    
    #subset dataframe by category
    column_name = dataframe[dataframe[category] == category_label][[labels_column]] 
    column_name['value_counts_column'] = column_name['labels'].map(column_name['labels'].value_counts())
    column_name = column_name.sort_values('value_counts_column', ascending=False).reset_index(drop=True)
    
    #shuffle data and transform to set
    # column_name = column_name.sample(frac=1).reset_index(drop=True)
    # column_name = column_name.values.flatten().tolist()
    # column_name = list(set(column_name))
    
    column_name = column_name.labels.values.flatten().tolist()
    print('number of non unique entries in column', len(column_name))
    
    #Create list of dictionaries of subgroups
    label_groups = {}
    for label_entry in column_name:
        counter = 0
        for label_group_key in label_groups.keys():
            if fuzz.ratio(label_entry, label_group_key) > 75:
                counter += 1
                if label_entry != label_group_key:
                    label_groups[label_group_key].append(label_entry)
        if counter == 0:
            label_groups[label_entry] = []
            
    return label_groups

In [279]:
food_dict = {}
for category_label in btype_data['food_category'].unique():
    food_dict[category_label] = group_labels(dataframe= btype_data,category='food_category', category_label=category_label)

number of non unique entries in column 30589
number of non unique entries in column 37187
number of non unique entries in column 8124
number of non unique entries in column 11121
number of non unique entries in column 11285
number of non unique entries in column 7230
number of non unique entries in column 10319
number of non unique entries in column 1520
number of non unique entries in column 26


In [280]:
label_sum = 0 
for category_label in btype_data['food_category'].unique():
    print(category_label, 'group size is', len(food_dict[category_label]))
    label_sum += len(food_dict[category_label])
    print('running sum', label_sum)

label_sum

Condiments & Snacks group size is 221
running sum 221
Produce group size is 201
running sum 422
Milk & Dairy group size is 24
running sum 446
Bread, Grains & Legumes group size is 96
running sum 542
Meat group size is 51
running sum 593
Beverages group size is 27
running sum 620
Meals group size is 193
running sum 813
Seafood group size is 43
running sum 856
Non-Food group size is 5
running sum 861


861

In [281]:
print('cod v code: same')
print(fuzz.ratio('code', 'cod'))
print(fuzz.QRatio('code', 'cod'))
print(fuzz.WRatio('code', 'cod'))
print(fuzz.partial_ratio('code', 'cod'))
print(fuzz.token_sort_ratio('code', 'cod'))
print(fuzz.token_set_ratio('code', 'cod'))

print('tilapia v talpia: same')
print(fuzz.ratio('tilapia', 'talpia'))
print(fuzz.QRatio('tilapia', 'talpia'))
print(fuzz.WRatio('tilapia', 'talpia'))
print(fuzz.partial_ratio('tilapia', 'talpia'))
print(fuzz.token_sort_ratio('tilapia', 'talpia'))
print(fuzz.token_set_ratio('tilapia', 'talpia'))

print('apple v pineapple: diff')
print(fuzz.ratio('apple', 'pineapple'))
print(fuzz.QRatio('apple', 'pineapple'))
print(fuzz.WRatio('apple', 'pineapple'))
print(fuzz.partial_ratio('apple', 'pineapple'))
print(fuzz.token_sort_ratio('apple', 'pineapple'))
print(fuzz.token_set_ratio('apple', 'pineapple'))

print('meatball v meat: diff')
print(fuzz.ratio('meatball', 'meat'))
print(fuzz.QRatio('meatball', 'meat'))
print(fuzz.WRatio('meatball', 'meat'))
print(fuzz.partial_ratio('meatball', 'meat'))
print(fuzz.token_sort_ratio('meatball', 'meat'))
print(fuzz.token_set_ratio('meatball', 'meat'))

print('pea v pear: diff')
print(fuzz.ratio('pea', 'pear'))
print(fuzz.QRatio('pea', 'pear'))
print(fuzz.WRatio('pea', 'pear'))
print(fuzz.partial_ratio('pea', 'pear'))
print(fuzz.token_sort_ratio('pea', 'pear'))
print(fuzz.token_set_ratio('pea', 'pear'))

cod v code: same
86
86
86
100
86
86
tilapia v talpia: same
77
77
77
73
77
77
apple v pineapple: diff
71
71
90
100
71
71
meatball v meat: diff
67
67
90
100
67
67
pea v pear: diff
86
86
86
100
86
86


In [282]:
dedupe_df = btype_data[['text','labels']]

There exists intersections in labels. For example almond's top level category can be Condiments & Snacks or Bread, Grains & Legumes.

In [299]:
def find_best_match(misspelled_value, choices):
    best_match = process.extractOne(misspelled_value, choices)
    return best_match[0]

In [284]:
meat = set(food_dict['Meat'].keys())
produce = set(food_dict['Produce'].keys())
condiments_snacks = set(food_dict['Condiments & Snacks'])
milk_dairy = set(food_dict['Milk & Dairy'])
beverages = set(food_dict['Beverages'])
bread_grain_legumes = set(food_dict['Bread, Grains & Legumes'])
meals = set(food_dict['Meals'])
seafood = set(food_dict['Seafood'])
nonfood = set(food_dict['Non-Food'])

In [285]:
def alter_food_set(food_set, add_list, rm_list):
    print('number of elements in food set before additions', len(food_set))
    
    food_set.update(add_list)
    
    food_set.difference_update(rm_list)
    
    print('number of elements in food set after additions', len(food_set))

In [309]:
food_dict['Produce']

{'pepper': ['peppers'],
 'lettuce': [],
 'apple': [],
 'tomato': ['tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tomatillo',
  'tamato',
  'tamato'],
 'carrot': ['carrots', 'ca

# 1. Meat

In [286]:
meat_add_list = []
meat_rm_list = []
alter_food_set(meat, meat_add_list, meat_rm_list)

number of elements in food set before additions 51
number of elements in food set after additions 51


# 2. Produce

In [287]:
produce_add_list = ['tomatillo',
                    'broccolini',
                    'broccoli rabe',
                    'vegetable cup',
                    'root vegetable',
                    'vegetable mix',
                    'pea',
                    'green pea',
                    'raspberry',
                    'chile',
                    'mustard greens',
                    'palm leaf',
                    'ti leaf']
produce_rm_list = ['mustard green']

alter_food_set(produce, produce_add_list, produce_rm_list)

number of elements in food set before additions 201
number of elements in food set after additions 213


# 3. Condiments Snacks

In [288]:
condiments_snacks_add_list = ['seasoning salt', 
                              'doughnut',
                              'cream', 'tomatillo',
                              'butter','sherbet',
                              'fruit cup','scone',
                              'ice cream cone',]
condiments_snacks_rm_list = []
alter_food_set(condiments_snacks, condiments_snacks_add_list, condiments_snacks_rm_list)

number of elements in food set before additions 221
number of elements in food set after additions 230


# 4. Milk Dairy

In [289]:
milk_dairy_add_list = ['cream']
milk_dairy_rm_list = []
alter_food_set(milk_dairy, milk_dairy_add_list, milk_dairy_rm_list)

number of elements in food set before additions 24
number of elements in food set after additions 25


# 5. Beverages

In [290]:
beverages_add_list = []
beverages_rm_list = []
alter_food_set(beverages, beverages_add_list, beverages_rm_list)

number of elements in food set before additions 27
number of elements in food set after additions 27


# 6. Bread Grain Legumes

In [291]:
bread_grain_legumes_add_list = ['scone']
bread_grain_legumes_rm_list = []
alter_food_set(bread_grain_legumes, bread_grain_legumes_add_list, bread_grain_legumes_rm_list)

number of elements in food set before additions 96
number of elements in food set after additions 97


# 7. Meals

In [292]:
meals_add_list = ['entree', 'pasta salad', 
                  'pretzel dog', 'baked potato', 
                  'egg substitute', 'egg alternative']
meals_rm_list = ['entrée']
alter_food_set(meals, meals_add_list, meals_rm_list)

number of elements in food set before additions 193
number of elements in food set after additions 198


# 8. Seafood

In [293]:
seafood_add_list = ['crawfish']
seafood_rm_list = []
alter_food_set(seafood, seafood_add_list, seafood_rm_list)

number of elements in food set before additions 43
number of elements in food set after additions 44


# 9. Non food

In [294]:
nonfood_add_list = []
nonfood_rm_list = []
alter_food_set(nonfood, nonfood_add_list, nonfood_rm_list)

number of elements in food set before additions 5
number of elements in food set after additions 5


In [295]:
basic_type = meat.union(produce, condiments_snacks, milk_dairy, beverages, bread_grain_legumes, meals, seafood, nonfood)

In [296]:
len(basic_type)

695

In [297]:
meat_df = btype_data[btype_data['food_category'] == 'Meat']
produce_df = btype_data[btype_data['food_category'] == 'Produce']
condiments_snacks_df = btype_data[btype_data['food_category'] == 'Condiments & Snacks']
milk_dairy_df = btype_data[btype_data['food_category'] == 'Milk & Dairy']
beverages_df = btype_data[btype_data['food_category'] == 'Beverages']
bread_grain_legumes_df = btype_data[btype_data['food_category'] == 'Bread, Grains & Legumes']
meals_df = btype_data[btype_data['food_category'] == 'Meals']
seafood_df = btype_data[btype_data['food_category'] == 'Seafood']
nonfood_df = btype_data[btype_data['food_category'] == 'Non-Food']

In [302]:
meat_df['correct_labels'] = meat_df['labels'].apply(lambda x: find_best_match(x, meat))
print('1')
produce_df['correct_labels'] = produce_df['labels'].apply(lambda x: find_best_match(x, produce))
print('2')
condiments_snacks_df['correct_labels'] = condiments_snacks_df['labels'].apply(lambda x: find_best_match(x, condiments_snacks))
print('3')
milk_dairy_df['correct_labels'] = milk_dairy_df['labels'].apply(lambda x: find_best_match(x, milk_dairy))
print('4')
beverages_df['correct_labels'] = beverages_df['labels'].apply(lambda x: find_best_match(x, beverages))
print('5')
bread_grain_legumes_df['correct_labels'] = bread_grain_legumes_df['labels'].apply(lambda x: find_best_match(x, bread_grain_legumes))
print('6')
meals_df['correct_labels'] = meals_df['labels'].apply(lambda x: find_best_match(x, meals))
print('7')
seafood_df['correct_labels'] = seafood_df['labels'].apply(lambda x: find_best_match(x, seafood))
print('8')
nonfood_df['correct_labels'] = nonfood_df['labels'].apply(lambda x: find_best_match(x, nonfood))
print('9')

/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_28059/3619392293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meat_df['correct_labels'] = meat_df['labels'].apply(lambda x: find_best_match(x, meat))


1


/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_28059/3619392293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  produce_df['correct_labels'] = produce_df['labels'].apply(lambda x: find_best_match(x, produce))


2


/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_28059/3619392293.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  condiments_snacks_df['correct_labels'] = condiments_snacks_df['labels'].apply(lambda x: find_best_match(x, condiments_snacks))


...

/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_28059/3619392293.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bread_grain_legumes_df['correct_labels'] = bread_grain_legumes_df['labels'].apply(lambda x: find_best_match(x, bread_grain_legumes))


6


/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_28059/3619392293.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meals_df['correct_labels'] = meals_df['labels'].apply(lambda x: find_best_match(x, meals))


7
8
9


/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_28059/3619392293.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seafood_df['correct_labels'] = seafood_df['labels'].apply(lambda x: find_best_match(x, seafood))
/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_28059/3619392293.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonfood_df['correct_labels'] = nonfood_df['labels'].apply(lambda x: find_best_match(x, nonfood))


In [303]:
print(len(meat_df))
print(len(produce_df))
print(len(condiments_snacks_df))
print(len(milk_dairy_df))
print(len(beverages_df))
print(len(bread_grain_legumes_df))
print(len(meals_df))
print(len(seafood_df))
print(len(nonfood_df))


11285
37187
30589
8124
7230
11121
10319
1520
26


In [304]:
basic_type_df = pd.concat([meat_df, produce_df, condiments_snacks_df, milk_dairy_df, beverages_df, bread_grain_legumes_df, meals_df, seafood_df, nonfood_df])
basic_type_df = shuffle(basic_type_df)
btype_dedupe_df = basic_type_df[['text', 'correct_labels']]
btype_dedupe_df.rename(columns={'correct_labels': 'labels'}, inplace=True)

/var/folders/6h/z9d4c_893fvfks6175tjrr940000gn/T/ipykernel_28059/1467162463.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btype_dedupe_df.rename(columns={'correct_labels': 'labels'}, inplace=True)


In [305]:
btype_dedupe_df

,text,labels
5393,SPICE PEPPER BLK GRND,spice
108986,MICRO CILANTRO 8 OZ,herb
21718,EGGPLANT EACH,eggplant
77199,Beef Sandwich Sausage White Cheddar,sandwich
38947,Juice Orange 4.2 oz Box,juice
...,...,...
60100,RICE BASMATI,rice
11862,"POTATO, FRENCH-FRY WEDGE 8 CUT SKIN-ON TFF EXT...",french fry
4133,"TURNOVER, BEEF PATTY 1.5 OZ COOKED FROZEN BULK...",turnover
81764,NADA CHIMI BEAN AND CHEESE WG IW,empanada


In [306]:
len(btype_dedupe_df.labels.unique())

695

In [307]:
dedupe_data_file_path = '/Users/anthonyhakim/DSI/good-food-purchasing/basic_type_deduped.csv'
btype_dedupe_df.to_csv(dedupe_data_file_path, index=False)